In [32]:
import ee
import pandas as pd
from datetime import datetime, timedelta
import time
import geemap
from datetime import datetime, timedelta


# Initialize Earth Engine
ee.Initialize()


In [46]:
data = pd.read_csv("C:\\Users\\admin-magstadt\\Desktop\\ADSB_data\\ATUData\\ATU_2017_2021_1mile_ALL.txt", sep=",", header=0)
data = data.head(1000)

datelist = data.UTCdateTim

long_list = data["Longitude1"]
lat_list = data["Latitude1"]

# Create an empty list to hold the Earth Engine Feature objects
features = []

# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    
    # Create a new Earth Engine Feature object for the row
    feature = ee.Feature(ee.Geometry.Point([row['Longitude1'], row['Latitude1']]), {'ID': str(row['Latitude1']) + ',' + str(row['Longitude1'])})
    # Add the Feature to the list of Features
    features.append(feature)
    
# Create an Earth Engine FeatureCollection from the list of Features
ee_fc_breach = ee.FeatureCollection(features)

In [47]:
dem = ee.Image("USGS/3DEP/10m").clip(ee_fc_breach.geometry().bounds())
topo_diversity = ee.Image("CSP/ERGo/1_0/US/topoDiversity").clip(ee_fc_breach.geometry().bounds())
physiography = ee.Image("CSP/ERGo/1_0/US/physiography").clip(ee_fc_breach.geometry().bounds())
physio_diversity = ee.Image("CSP/ERGo/1_0/US/physioDiversity").clip(ee_fc_breach.geometry().bounds())
mtpi = ee.Image("CSP/ERGo/1_0/US/mTPI").clip(ee_fc_breach.geometry().bounds())
lithology = ee.Image("CSP/ERGo/1_0/US/lithology").clip(ee_fc_breach.geometry().bounds())
landforms = ee.Image("CSP/ERGo/1_0/US/landforms").clip(ee_fc_breach.geometry().bounds())
chili = ee.Image("CSP/ERGo/1_0/US/CHILI").clip(ee_fc_breach.geometry().bounds())

# Extract data for each image at appropriate scale
terrain_fc = ee.Terrain.products(dem).sampleRegions(collection=ee_fc_breach, scale=10)

topo_diversity_fc = topo_diversity.sampleRegions(collection=ee_fc_breach, scale=90)
physiography_fc = physiography.sampleRegions(collection=ee_fc_breach, scale=90)
physio_diversity_fc = physio_diversity.sampleRegions(collection=ee_fc_breach, scale=270)
mtpi_fc = mtpi.sampleRegions(collection=ee_fc_breach, scale=270)
lithology_fc = lithology.sampleRegions(collection=ee_fc_breach, scale=90)
landforms_fc = landforms.sampleRegions(collection=ee_fc_breach, scale=10)
chili_fc = chili.sampleRegions(collection=ee_fc_breach, scale=10)

# Convert the elevation and terrain product data to Pandas DataFrames
terrain_df = geemap.ee_to_pandas(terrain_fc)

In [48]:
terrain_df

,ID,elevation,slope,hillshade,aspect
0,"46.914658,-114.079757",975.593750,0,180,233
1,"46.90025,-114.064118",963.997009,0,180,87
2,"30.332868,-82.542972",39.405899,1,181,339
3,"30.332652,-82.54137299999998",39.874519,1,181,335
4,"30.33101,-82.54291999999998",39.294449,0,181,325
...,...,...,...,...,...
992,"39.424,-117.9016",1748.727539,17,134,119
993,"39.422272,-117.85703",1795.014771,8,159,64
994,"39.715745,-119.54216",1629.611084,6,173,24
995,"39.705518,-119.525122",1791.251343,29,173,175


In [ ]:
variablelist = ["pr","rmax","rmin","sph","srad","th","tmmn","tmmx","vs","erc","eto","bi","fm100","fm1000","etr","vpd"]

totaldays = 1


for i in range(len(variablelist)):
    print(i)
    
    #DAYMET_TMAX = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').select(variablelist[i]).filterDate(start_date, start_date + timedelta(days=totaldays))
    DAYMET_TMAX = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').select(variablelist[i]).filterDate(start_date, start_date + timedelta(days=totaldays))

    DAYMET_TMAX_IMG = DAYMET_TMAX.toBands()

    bandNames = DAYMET_TMAX_IMG.bandNames()

    out_csv= f'C:\\Users\\admin-magstadt\\Desktop\\ATU_terrain_6_20_2023\\{variablelist[i]}_data.csv'

    geemap.extract_values_to_points(ee_fc, DAYMET_TMAX_IMG, out_csv)
    
print("finish")


In [31]:
# terrain held
# Load the 3DEP dataset and filter by the extent of the points
dem = ee.Image("USGS/3DEP/10m").clip(ee_fc_held.geometry().bounds())
topo_diversity = ee.Image("CSP/ERGo/1_0/US/topoDiversity").clip(ee_fc_held.geometry().bounds())
physiography = ee.Image("CSP/ERGo/1_0/US/physiography").clip(ee_fc_held.geometry().bounds())
physio_diversity = ee.Image("CSP/ERGo/1_0/US/physioDiversity").clip(ee_fc_held.geometry().bounds())
mtpi = ee.Image("CSP/ERGo/1_0/US/mTPI").clip(ee_fc_held.geometry().bounds())
lithology = ee.Image("CSP/ERGo/1_0/US/lithology").clip(ee_fc_held.geometry().bounds())
landforms = ee.Image("CSP/ERGo/1_0/US/landforms").clip(ee_fc_held.geometry().bounds())
chili = ee.Image("CSP/ERGo/1_0/US/CHILI").clip(ee_fc_held.geometry().bounds())

# Extract data for each image at appropriate scale
terrain_fc = ee.Terrain.products(dem).sampleRegions(collection=ee_fc_held, scale=10)

topo_diversity_fc = topo_diversity.sampleRegions(collection=ee_fc_held, scale=90)
physiography_fc = physiography.sampleRegions(collection=ee_fc_held, scale=90)
physio_diversity_fc = physio_diversity.sampleRegions(collection=ee_fc_held, scale=270)
mtpi_fc = mtpi.sampleRegions(collection=ee_fc_held, scale=270)
lithology_fc = lithology.sampleRegions(collection=ee_fc_held, scale=90)
landforms_fc = landforms.sampleRegions(collection=ee_fc_held, scale=10)
chili_fc = chili.sampleRegions(collection=ee_fc_held, scale=10)

# Convert the elevation and terrain product data to Pandas DataFrames
terrain_df = geemap.ee_to_pandas(terrain_fc)

topo_diversity_df = geemap.ee_to_pandas(topo_diversity_fc)
physiography_df = geemap.ee_to_pandas(physiography_fc)
physio_diversity_df = geemap.ee_to_pandas(physio_diversity_fc)
mtpi_df = geemap.ee_to_pandas(mtpi_fc)
lithology_df = geemap.ee_to_pandas(lithology_fc)
landforms_df = geemap.ee_to_pandas(landforms_fc)
chili_df = geemap.ee_to_pandas(chili_fc)

# Rename columns to appropriate names
topo_diversity_df = topo_diversity_df.rename(columns={'constant': 'topo_diversity'})
physiography_df = physiography_df.rename(columns={'constant': 'physiography'})
physio_diversity_df = physio_diversity_df.rename(columns={'b1': 'physio_diversity'})
mtpi_df = mtpi_df.rename(columns={'elevation': 'mtpi'})
lithology_df = lithology_df.rename(columns={'b1': 'lithology'})
landforms_df = landforms_df.rename(columns={'constant': 'landforms'})
chili_df = chili_df.rename(columns={'constant': 'chili'})

##########################
##### Held Weather #######
##########################

start_date_str = str(datelist[i])
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = start_date + timedelta(days=1)

era5 = ee.ImageCollection("ECMWF/ERA5/HOURLY").select(['u_component_of_wind_10m', 'v_component_of_wind_10m', 
                                                  'dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature',
                                                  'soil_temperature_level_1', 'volumetric_soil_water_layer_1',
                                                  'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3',
                                                  'volumetric_soil_water_layer_4', 'forecast_albedo',
                                                  'surface_latent_heat_flux', 'surface_net_solar_radiation',
                                                  'surface_net_thermal_radiation',
                                                  'potential_evaporation',
                                                  'total_precipitation', 'leaf_area_index_high_vegetation',
                                                  'leaf_area_index_low_vegetation']).filterDate(start_date, end_date)


era5mean_fc = era5.mean().reduceRegions(collection=ee_fc_held, reducer=ee.Reducer.mean(), scale=11132)
era5mean_df = geemap.ee_to_pandas(era5mean_fc, timeout=600)

#print(era5mean_df)


##########################
### END Held Weather #####
##########################

##########################
### GridMet Weather ######
##########################

GRIDMET = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').select(["pr","rmax","rmin","sph","srad",
                                                             "th","tmmn","tmmx","vs","erc",
                                                             "eto","bi","fm100","fm1000",
                                                             "etr","vpd"]).filterDate(start_date, end_date)


GRIDMETmean_fc = GRIDMET.mean().reduceRegions(collection=ee_fc_held, reducer=ee.Reducer.mean(), scale=4638.3)
GRIDMETmean_df = geemap.ee_to_pandas(GRIDMETmean_fc, timeout=600)


##########################
### END GridMet Weather ##
##########################

df = pd.concat([terrain_df, topo_diversity_df.iloc[:, 1], physiography_df.iloc[:, 1], physio_diversity_df.iloc[:, 1], 
                mtpi_df.iloc[:, 1], lithology_df.iloc[:, 1], landforms_df.iloc[:, 1], chili_df.iloc[:, 1], era5mean_df, GRIDMETmean_df], axis=1)

NameError: name 'i' is not defined